In [1]:
#Initialization
project_name = 'yolov5_10m_5by5'
epochs_no = 500

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

# Install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
!pip install roboflow

# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

# Imports
import torch
import sys
import shutil
import datetime
from datetime import datetime
from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets
from roboflow import Roboflow

#%cd /content/yolov5

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

######################CHANGE##################
# Download and setup folder structure for the Aerial Imagery
rf = Roboflow(api_key="2vNzUkqiG7v3Cq182eZh")
project = rf.workspace("aerial-imagery-small-objects").project("aerial_imagery_10m")
dataset = project.version(10).download("yolov5")



# Define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

# Customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Setup complete. Using torch 1.12.1+cu116 _CudaDeviceProperties(name='NVIDIA RTX A6000', major=8, minor=6, total_memory=48676MB, multi_processor_count=84)
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to aerial_imagery_10m-10 in yolov5pytorch:: 100%|██████████| 2152/2152 [00:05<00:00, 375.34it/s]


In [2]:
%cd /notebooks/yolov5/
!python train.py --img 640 --batch 16 --epochs 500 --data {dataset.location}/data.yaml  --weights 'yolov5s.pt' --name {project_name}

/notebooks/yolov5
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=/notebooks/yolov5/aerial_imagery_10m-10/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_10m_5by5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 126 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-72-g

In [6]:
%cd /notebooks/yolov5/
!python val.py --img 1280 --weights /notebooks/yolov5/runs/train/yolov5_10m_5by5/weights/best.pt --data {dataset.location}/data.yaml --task test --iou-thres 0.45

/notebooks/yolov5
val: data=/notebooks/yolov5/aerial_imagery_10m-10/data.yaml, weights=['/notebooks/yolov5/runs/train/yolov5_10m_5by5/weights/best.pt'], batch_size=32, imgsz=1280, conf_thres=0.001, iou_thres=0.45, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-72-g064365d Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A6000, 48677MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /notebooks/yolov5/aerial_imagery_10m-10/test/labels.cache... 106 
                 Class     Images  Instances          P          R      mAP50   
                   all        106        122      0.697      0.519      0.611      0.395
  clear_plastic_bottle        106         29      0.742       0.69       0.76      0.433
             drink_can        106